In [1]:
import numpy as np

np.random.seed(1)


In [7]:
def f1(X):
    return sum([x**2 for x in X])

def f2(X):
    return 10*len(X) + sum([(x**2 - 10*np.cos(2*np.pi*x)) for x in X])

def getX(n,d):
    X = []
    for i in range(n):
        xs = np.random.uniform(-5.13, 5.13, d)
        X.append(xs)
    return X

def update_v(v,a1,a2,w,x,p,g):
    v = w*v + a1*r1*(p-x)+a2*r2*(g-x)
    return v

In [22]:
def pso_f1(N,d,X,T,a1,a2,w,r1=0.2,r2=0.18):
    V = [[0]*d]*N
    f1_xs = [f1(x) for x in X]
    p1 = g1 = [x for x in X[f1_xs.index(min(f1_xs))]]

    f1_p = f1(p1)
    f1_g = f1(g1)
    for t in range(T):
        f1_xs = [f1(x) for x in X]
        p1 = [x for x in X[f1_xs.index(min(f1_xs))]]
        f1_p = f1(p1)
        for i in range(N):
            V[i] = update_v(V[i],a1,a2,w,X[i],p1,g1)
            X[i] = X[i]+V[i]
            for j in range(d):
                if X[i][j] < -5.12:
                    X[i][j] = -5.12
                elif X[i][j] > 5.12:
                    X[i][j] = 5.12
            f1_x = f1(X[i]) 
            if f1_x < f1_p:
                f1_p = f1_x
                p1 = X[i]
                if f1_x < f1_g:
                    f1_g = f1_x
                    g1 = X[i]
                   # print(g1,t)
        #print(X)            
    return g1,f1_g,p1,f1_p


In [30]:
def pso_f2(N,d,X,T,a1,a2,w,r1=0.2,r2=0.18):
    V = [[0]*d]*N
    f_xs = [f2(x) for x in X]
    p = g = [x for x in X[f_xs.index(min(f_xs))]]

    f_p = f2(p)
    f_g = f2(g)
    for t in range(T):
        f_xs = [f1(x) for x in X]
        p = [x for x in X[f_xs.index(min(f_xs))]]
        f_p = f2(p1)
        for i in range(N):
            V[i] = update_v(V[i],a1,a2,w,X[i],p,g)
            X[i] = X[i]+V[i]
            for j in range(d):
                if X[i][j] < -5.12:
                    X[i][j] = -5.12
                elif X[i][j] > 5.12:
                    X[i][j] = 5.12
            f_x = f2(X[i]) 
            if f_x < f_p:
                f_p = f_x
                p = X[i]
                if f_x < f_g:
                    f_g = f_x
                    g = X[i]

    return g,f_g,p,f_p

In [ ]:
f2_x = [f2(x) for x in X]
p2 = g2 = [x for x in X[f2_x.index(min(f2_x))]]

In [45]:
N = 10000
d = 3


a1 = 0.5
a2 = 0.5
w = 1
r1 = np.random.rand()
r2 = np.random.rand()
T= 20
X = getX(N,d)

print(pso_f1(N,d,X,T,a1,a2,w,r1,r2))

(array([-0.00297826, -0.00229721, -0.00622328]), 5.2876410943062985e-05, array([0.2103041 , 0.01767709, 0.0748157 ]), 0.050137683398681486)


In [46]:
print(pso_f2(N,d,X,T,a1,a2,w,r1,r2))

(array([ 0.01433547,  0.00907188, -0.00337403]), 0.059324781386223435, array([-1.01611881, -0.03055853,  1.00279962]), 2.2755910153083647)


[array([-1.10835112,  0.77676047]),
 array([-2.84123452, -1.15928533]),
 array([-2.09279235, -2.13675915]),
 array([-1.60868444, -0.6822641 ]),
 array([-0.62882139,  0.21729831]),
 array([-0.42939357,  0.89309271]),
 array([-1.42896958,  1.79098838]),
 array([-2.26367657,  0.82442558]),
 array([-0.43818975,  0.30412609]),
 array([-1.67196229, -1.3866776 ]),
 array([1.40185109, 2.202314  ]),
 array([-0.86651361,  0.91380844]),
 array([1.75395928, 1.85539431]),
 array([-1.92957   , -2.12386319]),
 array([-1.53490964,  1.77875742]),
 array([-1.86764934, -0.34863516]),
 array([2.13046079, 0.17296755]),
 array([ 0.89509804, -0.84014169]),
 array([0.87007313, 1.57619654]),
 array([-2.26928294,  1.1829552 ]),
 array([2.32953196, 1.17374498]),
 array([-1.02002881,  1.36511971]),
 array([-1.84493791, -0.22395293]),
 array([ 1.90387246, -0.94208807]),
 array([-0.98590308, -1.70354142]),
 array([-2.26207267,  0.85102916]),
 array([-1.34035094, -1.07273579]),
 array([-0.03727094, -2.0604042 ]),
 a

In [21]:
for x in X:
    if 0.00147901 in x:
        print(1111111111111)

print(0)

0


In [11]:
fs = [f1(x) for x in X]

Question1.2